<a href="https://colab.research.google.com/github/aviax1/AE1/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**dependencies**

In [13]:
# used snniped from https://github.com/L1aoXingyu/pytorch-beginner/
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorflow.keras.datasets import mnist
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np

**initial**

In [14]:
(xtrain,ytrain), (xtest,ytest) = mnist.load_data()
num_epochs=200      #
batch_size = 20    #
image_size=784      #
hidden_size=128     #
lv_size = 64        # Latent Variable 
learning_rate=1e-4  #
cret = nn.MSELoss() # criterion

**build model**

In [15]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, image_size),nn.ReLU(True), nn.Linear(image_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size), nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),nn.Linear(hidden_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, image_size), nn.Tanh())

    def forward(self, x):
        return self.decoder(self.encoder(x))

**model setting**

In [16]:
model = autoencoder()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

class DigitDataSet(Dataset):
  def __init__(self, dataset):
      self.dataset = dataset
      self.transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()
      return self.transform( self.dataset[idx,:,:])


def visual_epoch(model,dataloader):
  for data in dataloader:
      input_imgs = data
      visual_batch(input_imgs)
      imgs = Variable(input_imgs.view(input_imgs.size(0), -1))
      output_imgs = model(imgs)

      images_in_row=5
      dis_images = input_imgs[0:images_in_row,:,:,:]
      for i in range(images_in_row):
        ax = plt.subplot(2, images_in_row, i + 1)
        plt.imshow(dis_images[i].reshape(28, 28))
        plt.gray()
       

**train model by digit**

In [17]:
for by_digit in range(10):
  print("*****\nstart traning Model for digit " +str(by_digit) +"\n")
  dataloader = DataLoader(DigitDataSet(xtrain[ytrain==by_digit]), batch_size=batch_size,shuffle=True, num_workers=4)
  
  for epoch in range(1):
    for data in dataloader:
      input_imgs = data
      imgs = Variable(input_imgs.view(input_imgs.size(0), -1))
      output_imgs = model(imgs)
      loss = cret(output_imgs, imgs)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    visual_epoch(model,dataloader)
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, num_epochs, loss.data))
    

  torch.save(model.state_dict(), './ae_'+by_digit+'.pth')
  print("\nfinish traning Model Number " +str(by_digit) +"\n")
  print("*****\n")

*****
start traning Model for digit 0



UnboundLocalError: ignored